In [1]:
import hw1_v2 as hw1
import yahoofinancials as yfs
import yfinance as yf
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve

## Companies: GE, AAPL, TSLA, MSFT,  GS

### 1. Take credit issuers you were assigned and perform the following steps:
- Find their public ratings from major providers – lookup PD’s for one year, also look up cumulative PD’s for each of the ratings over 5-year time periods

- Calculate their Altman’s Z scores based on available data – does Z score agree with public ratings? What would you change based on this information?

- Perform Merton model analysis on each of the issuers, calculate their DD’s and implied PD’s over 5 years

- Obtain credit spreads for each of the issuers, calculate and draw default intensity step term structure (using Hull approach) – is it different from above steps, why?

- Calculate implied JT-model π’s from the generic issuer with the same rating (i.e. look at generic 5-yr spread for issuer in the same rating category, solve back for π using 5-year cumulative transition matrix), apply those π’s to your issuers PD’s calculated from Hull above, then – what should their 5-year credit spreads be? Are they consistent with the market pricing? Why yes, or why no?

### 2. Compare PD’s from each of the steps above; write about how you would you use these estimates together
### 3. Write up your findings in the clear concise summary

## Altman's Z
- X1 – Working Capital / Total Assets
- X2 – Retained Earnings since Inception / Total Assets 
- X3 – Profit Before Interest and Tax / Total Assets
- X4 – Market Value of Equity / Book Value of Total Debt 
- X5 – Sales / Total Asset

Working Capital = Current Asset - Current Liabilities


The Z-score then leads to assessment of bankruptcy risk as Low (Z > 2.8), Moderate/Uncertain (1.8 < Z < 2.8), or Very High (Z < 1.8)


In [2]:
def get_equity_vol(ticker,start_date,end_date,interval):
    '''param ticker: could be list or string, could be one or multiple
       param start_date: in format 'YYYY-MM-DD'
       param end_date: in format 'YYYY-MM-DD'
       param interval: valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
    '''
    
    return yf.download(tickers=ticker,start=start_date, end=end_date,interval=interval)[
        'Adj Close'].pct_change().std()*np.sqrt(252)
    

In [5]:
tickers=["AAPL","GE","TSLA","MSFT","GS"]
start="2018-10-01"
end="2019-10-01"
interval='1d'

vol = get_equity_vol(ticker=tickers,start_date=start,end_date=end,interval=interval)

[*********************100%***********************]  5 of 5 downloaded


In [6]:
vol

AAPL    0.320813
GE      0.462401
GS      0.280134
MSFT    0.269190
TSLA    0.518855
dtype: float64

**Note**:

- Used 1 yr of stock prices (from 2018-10-1 to 2019-10-1)
- Annulized volatility



# 1. AAPL


In [ ]:
## 1.0 

In [7]:

aapl_totalAsset = yfs.YahooFinancials('AAPL').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["AAPL"][0]['2018-09-29']['totalAssets'];

aapl_totalCurrLib = yfs.YahooFinancials('AAPL').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["AAPL"][0]['2018-09-29']['totalCurrentLiabilities'];

aapl_totalCurrAsset = yfs.YahooFinancials('AAPL').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["AAPL"][0]['2018-09-29']['totalCurrentAssets'];

aapl_RetainedEarning = yfs.YahooFinancials('AAPL').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["AAPL"][0]['2018-09-29']['retainedEarnings'];

aapl_EBIT = yfs.YahooFinancials('AAPL').get_financial_stmts('annual','income')\
["incomeStatementHistory"]["AAPL"][0]['2018-09-29']['ebit']

aapl_mkt_val = yfs.YahooFinancials('AAPL').get_market_cap()

aapl_sales = yfs.YahooFinancials('AAPL').get_total_revenue()


In [8]:
# Working Capital
aapl_working_cap = aapl_totalCurrAsset-aapl_totalCurrLib
aapl_working_cap

14473000000

In [9]:
# Book Value of Debt

aapl_longtermDebt = yfs.YahooFinancials('AAPL').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["AAPL"][0]['2018-09-29']['longTermDebt'];

aapl_accountsPayable = yfs.YahooFinancials('AAPL').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["AAPL"][0]['2018-09-29']['accountsPayable'];

aapl_book_val_debt = aapl_accountsPayable+aapl_longtermDebt+aapl_totalCurrLib

**Note:**
- Book Value of Debt = Long Term Debt + Notes Payable + Current Portion of Long-Term Debt
- I used _accountsPayable_ as  _Notes Payable_
- _Total current debt_ as _Curent Portion of Long-Term Debt_

## 1.1 Altman's Z

In [10]:
aapl_altman_z=hw1.calc_altman_z(total_assets=aapl_totalAsset,
                                working_capital=aapl_working_cap,
                                retained_earnings=aapl_RetainedEarning,
                                ebit=aapl_EBIT,
                                mv_equity=aapl_mkt_val,
                                sales=aapl_sales,
                               book_value_debt=aapl_book_val_debt)
aapl_altman_z

5.378392427067647

Z > 2.8, bankruptcy risk is low.

## 1.2 Merton

In [11]:
aapl_totalLib = yfs.YahooFinancials('AAPL').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["AAPL"][0]['2018-09-29']['totalLiab'];


In [38]:
dict_profile = {}
for num in range(1, 6):
    aapl_DD_Merton,aapl_PD_Merton =hw1.calc_prob_default(assets=aapl_totalAsset,
                                                         equity=aapl_mkt_val,
                                                         debt=aapl_totalLib,
                                                         t=num,
                                                         equity_vol=vol['AAPL'],
                                                         rf=0.0173)
    dict_profile.update({num: (aapl_DD_Merton, aapl_PD_Merton)})
    
dict_profile

{1: (1.6436963042864114, 0.050119474869201264),
 2: (-0.4077213226851055, 0.658260859360561),
 3: (-1.6900280421311717, 0.9544887050353148),
 4: (-2.9572599582868384, 0.9984480684240471),
 5: (-4.312975848772996, 0.9999919464140388)}

# 2.  GE

In [13]:
ge_totalAsset = yfs.YahooFinancials('GE').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GE"][0]['2018-12-31']['totalAssets'];

ge_totalCurrLib = yfs.YahooFinancials('GE').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GE"][0]['2018-12-31']['totalCurrentLiabilities'];

ge_totalCurrAsset = yfs.YahooFinancials('GE').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GE"][0]['2018-12-31']['totalCurrentAssets'];

ge_RetainedEarning = yfs.YahooFinancials('GE').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GE"][0]['2018-12-31']['retainedEarnings'];

ge_EBIT = yfs.YahooFinancials('GE').get_financial_stmts('annual','income')\
["incomeStatementHistory"]["GE"][0]['2018-12-31']['ebit']

ge_mkt_val = yfs.YahooFinancials('GE').get_market_cap()

ge_sales = yfs.YahooFinancials('GE').get_total_revenue()


In [14]:
# Working Capital
ge_working_cap = ge_totalCurrAsset-ge_totalCurrLib
ge_working_cap

46037000000

In [15]:
# Book Value of Debt

ge_longtermDebt = yfs.YahooFinancials('GE').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GE"][0]['2018-12-31']['longTermDebt'];

ge_accountsPayable = yfs.YahooFinancials('GE').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GE"][0]['2018-12-31']['accountsPayable'];

ge_book_val_debt = ge_accountsPayable+ge_longtermDebt+ge_totalCurrLib

## 1.1 Altman's Z

In [16]:
ge_altman_z=hw1.calc_altman_z(total_assets=ge_totalAsset,
                                working_capital=ge_working_cap,
                                retained_earnings=ge_RetainedEarning,
                                ebit=ge_EBIT,
                                mv_equity=ge_mkt_val,
                                sales=ge_sales,
                                book_value_debt=ge_book_val_debt)
ge_altman_z

1.408276598033333

Z < 1.8, bankruptcy risk is high.

## 1.2 Merton

In [17]:
ge_totalLib = yfs.YahooFinancials('GE').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GE"][0]['2018-12-31']['totalLiab'];

ge_volume = yfs.YahooFinancials('GE').get_current_volume()

In [19]:
ge_dict_profile = {}
for num in range(1, 6):
    ge_DD_Merton,ge_PD_Merton =hw1.calc_prob_default(assets=ge_totalAsset,
                                                         equity=ge_mkt_val,
                                                         debt=ge_totalLib,
                                                         t=num,
                                                         equity_vol=vol['GE'],
                                                         rf=0.0173)
    ge_dict_profile.update({num: (ge_DD_Merton, ge_PD_Merton)})
    
ge_dict_profile

{1: (1.0620396313512823, 0.14410884696662402),
 2: (0.10919137891294095, 0.4565253490276033),
 3: (-0.3597120589277134, 0.6404687634184646),
 4: (-0.786600573272322, 0.7842421382072979),
 5: (-1.258718364884862, 0.8959339621713638)}

# 3. TSLA

In [20]:
tsla_totalAsset = yfs.YahooFinancials('TSLA').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["TSLA"][0]['2018-12-31']['totalAssets'];

tsla_totalCurrLib = yfs.YahooFinancials('TSLA').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["TSLA"][0]['2018-12-31']['totalCurrentLiabilities'];

tsla_totalCurrAsset = yfs.YahooFinancials('TSLA').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["TSLA"][0]['2018-12-31']['totalCurrentAssets'];

tsla_RetainedEarning = yfs.YahooFinancials('TSLA').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["TSLA"][0]['2018-12-31']['retainedEarnings'];

tsla_EBIT = yfs.YahooFinancials('TSLA').get_financial_stmts('annual','income')\
["incomeStatementHistory"]["TSLA"][0]['2018-12-31']['ebit']

tsla_mkt_val = yfs.YahooFinancials('TSLA').get_market_cap()

tsla_sales = yfs.YahooFinancials('TSLA').get_total_revenue()

In [21]:
# Working Capital
tsla_working_cap = tsla_totalCurrAsset-tsla_totalCurrLib
tsla_working_cap

-1685828000

In [22]:
# Book Value of Debt

tsla_longtermDebt = yfs.YahooFinancials('TSLA').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["TSLA"][0]['2018-12-31']['longTermDebt'];

tsla_accountsPayable = yfs.YahooFinancials('TSLA').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["TSLA"][0]['2018-12-31']['accountsPayable'];

tsla_book_val_debt = tsla_accountsPayable+tsla_longtermDebt+tsla_totalCurrLib

## 1.1 Altman's Z

In [23]:
tsla_altman_z=hw1.calc_altman_z(total_assets=tsla_totalAsset,
                                working_capital=tsla_working_cap,
                                retained_earnings=tsla_RetainedEarning,
                                ebit=tsla_EBIT,
                                mv_equity=tsla_mkt_val,
                                sales=tsla_sales,
                                book_value_debt=tsla_book_val_debt)
tsla_altman_z

2.2313243245613523

1.8 < Z < 2.8, bankruptcy risk is moderate.

## 1.2 Merton

In [24]:
tsla_totalLib = yfs.YahooFinancials('TSLA').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["TSLA"][0]['2018-12-31']['totalLiab'];

tsla_volume = yfs.YahooFinancials('TSLA').get_current_volume()

In [25]:
tsla_dict_profile = {}
for num in range(1, 6):
    tsla_DD_Merton,tsla_PD_Merton =hw1.calc_prob_default(assets=tsla_totalAsset,
                                                         equity=tsla_mkt_val,
                                                         debt=tsla_totalLib,
                                                         t=num,
                                                         equity_vol=vol['TSLA'],
                                                         rf=0.0173)
    tsla_dict_profile.update({num: (tsla_DD_Merton, tsla_PD_Merton)})
    
tsla_dict_profile

{1: (1.3306244441279167, 0.09165630761486726),
 2: (-0.3482650148285822, 0.6361794173555051),
 3: (-1.4077421080115862, 0.9203962758374551),
 4: (-2.459657384444017, 0.993046514654468),
 5: (-3.5963158412376783, 0.9998386220551745)}

# 4. MSFT

In [26]:
msft_totalAsset = yfs.YahooFinancials('MSFT').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["MSFT"][0]['2019-06-30']['totalAssets'];

msft_totalCurrLib = yfs.YahooFinancials('MSFT').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["MSFT"][0]['2019-06-30']['totalCurrentLiabilities'];

msft_totalCurrAsset = yfs.YahooFinancials('MSFT').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["MSFT"][0]['2019-06-30']['totalCurrentAssets'];

msft_RetainedEarning = yfs.YahooFinancials('MSFT').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["MSFT"][0]['2019-06-30']['retainedEarnings'];

msft_EBIT = yfs.YahooFinancials('MSFT').get_financial_stmts('annual','income')\
["incomeStatementHistory"]["MSFT"][0]['2019-06-30']['ebit']

msft_mkt_val = yfs.YahooFinancials('MSFT').get_market_cap()
msft_sales = yfs.YahooFinancials('MSFT').get_total_revenue()


In [27]:
# Working Capital
msft_working_cap = msft_totalCurrAsset-msft_totalCurrLib
msft_working_cap

106132000000

In [28]:
# Book Value of Debt

msft_longtermDebt = yfs.YahooFinancials('MSFT').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["MSFT"][0]['2019-06-30']['longTermDebt'];

msft_accountsPayable = yfs.YahooFinancials('MSFT').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["MSFT"][0]['2019-06-30']['accountsPayable'];

msft_book_val_debt = msft_accountsPayable+msft_longtermDebt+msft_totalCurrLib

## 1.1 Altman's Z

In [29]:
msft_altman_z=hw1.calc_altman_z(total_assets=msft_totalAsset,
                                working_capital=msft_working_cap,
                                retained_earnings=msft_RetainedEarning,
                                ebit=msft_EBIT,
                                mv_equity=msft_mkt_val,
                                sales=msft_sales,
                                book_value_debt=msft_book_val_debt)
msft_altman_z

8.585369987408502

Z > 2.8, bankruptcy risk is low.

## 1.2 Merton

In [30]:
msft_totalLib = yfs.YahooFinancials('MSFT').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["MSFT"][0]['2019-06-30']['totalLiab'];

msft_volume = yfs.YahooFinancials('MSFT').get_current_volume()

In [31]:
msft_dict_profile = {}
for num in range(1, 6):
    msft_DD_Merton,msft_PD_Merton =hw1.calc_prob_default(assets=msft_totalAsset,
                                                         equity=msft_mkt_val,
                                                         debt=msft_totalLib,
                                                         t=num,
                                                         equity_vol=vol['MSFT'],
                                                         rf=0.0173)
    msft_dict_profile.update({num: (msft_DD_Merton, msft_PD_Merton)})
    
msft_dict_profile

{1: (1.8325144467018948, 0.0334374056979105),
 2: (-0.3995519188411554, 0.6552567119157138),
 3: (-1.7717519529830708, 0.9617821292874051),
 4: (-3.119119678395128, 0.9990930386629133),
 5: (-4.555649090432154, 0.9999973887970963)}

# 5. GS

In [32]:
gs_totalAsset = yfs.YahooFinancials('GS').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GS"][0]['2018-12-31']['totalAssets'];

gs_totalCurrLib = yfs.YahooFinancials('GS').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GS"][0]['2018-12-31']['totalCurrentLiabilities'];

gs_totalCurrAsset = yfs.YahooFinancials('GS').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GS"][0]['2018-12-31']['totalCurrentAssets'];

gs_RetainedEarning = yfs.YahooFinancials('GS').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GS"][0]['2018-12-31']['retainedEarnings'];

gs_EBIT = yfs.YahooFinancials('GS').get_financial_stmts('annual','income')\
["incomeStatementHistory"]["GS"][0]['2018-12-31']['ebit']

gs_mkt_val = yfs.YahooFinancials('GS').get_market_cap()

gs_sales = yfs.YahooFinancials('GS').get_total_revenue()

In [33]:
# Working Capital
gs_working_cap = gs_totalCurrAsset-gs_totalCurrLib
gs_working_cap

427413000000

In [34]:
# Book Value of Debt

gs_longtermDebt = yfs.YahooFinancials('GS').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GS"][0]['2018-12-31']['longTermDebt'];

gs_accountsPayable = yfs.YahooFinancials('GS').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GS"][0]['2018-12-31']['accountsPayable'];

gs_book_val_debt = gs_accountsPayable+gs_longtermDebt+gs_totalCurrLib

## 1.1 Altman's Z

In [35]:
gs_altman_z=hw1.calc_altman_z(total_assets=gs_totalAsset,
                                working_capital=gs_working_cap,
                                retained_earnings=gs_RetainedEarning,
                                ebit=gs_EBIT,
                                mv_equity=gs_mkt_val,
                                sales=gs_sales,
                                book_value_debt=gs_book_val_debt)
gs_altman_z

0.8067534196548376

Z < 1.8, bankruptcy risk is high.

## 1.2 Merton

In [36]:
gs_totalLib = yfs.YahooFinancials('GS').get_financial_stmts('annual','balance')\
["balanceSheetHistory"]["GS"][0]['2018-12-31']['totalLiab'];

gs_volume = yfs.YahooFinancials('GS').get_current_volume()

In [37]:
gs_dict_profile = {}
for num in range(1, 6):
    gs_DD_Merton,gs_PD_Merton =hw1.calc_prob_default(assets=gs_totalAsset,
                                                         equity=gs_mkt_val,
                                                         debt=gs_totalLib,
                                                         t=num,
                                                         equity_vol=vol['GS'],
                                                         rf=0.0173)
    gs_dict_profile.update({num: (gs_DD_Merton, gs_PD_Merton)})
    
gs_dict_profile

{1: (1.155059478005004, 0.12403303535336546),
 2: (0.382425182068711, 0.35107300735669256),
 3: (0.09764634483583268, 0.46110656123188715),
 4: (-0.11281220261028013, 0.5449102780894848),
 5: (-0.32414764063201496, 0.6270868698236023)}